In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import collections
import warnings
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import History
from scipy.cluster.hierarchy import linkage
warnings.filterwarnings('ignore')

In [ ]:
!pip install Boruta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load RNA dataset
rna_df = pd.read_csv("/content/drive/MyDrive/Jaya New Files/GDC_rnaseq_fpkm_unstranded_v4.csv",index_col=0)
#load clinical datset
clinical_df = pd.read_csv("/content/drive/MyDrive/Jaya New Files/BRCA-subtypes-combined-tumor-samples-only-gdc-tcga-refined.csv")

In [ ]:
#rna_df = rna_df.set_index('sample')
#rna_df = rna_df.fillna(0)

#samples = rna_df.columns.values
#samples = [i for i in samples if i.endswith('01')]

#rna_df2 = rna_df[samples]

In [ ]:
rna_df = rna_df.fillna(0)
rna_df.head()

TCGA-3C-AAAU  TCGA-3C-AALI  TCGA-3C-AALJ  TCGA-3C-AALK  \
ENSG00000000003.15        2.5997        2.5597       13.1981       12.8083   
ENSG00000000005.6         0.0245        0.0376        1.6187        0.0530   
ENSG00000000419.13       28.0941       41.8377       48.7619       22.4704   
ENSG00000000457.14        2.5028        9.8346        2.5366        3.7386   
ENSG00000000460.17        1.0286        2.2798        1.8393        1.5078   

                    TCGA-4H-AAAK  TCGA-5L-AAT0  TCGA-5T-A9QA  TCGA-A1-A0SB  \
ENSG00000000003.15       12.1947       19.1111       10.5157       19.6132   
ENSG00000000005.6         0.2186        0.1020        0.0206        8.7767   
ENSG00000000419.13       25.6404       22.9089       55.6637       21.2168   
ENSG00000000457.14        3.1496        2.9714        5.1147        3.1036   
ENSG00000000460.17        1.3185        1.2210        2.3577        1.2107   

                    TCGA-A1-A0SD  TCGA-A1-A0SE  ...  TCGA-UL-AAZ6  \
ENSG00000000003.15        9.2472       18.7699  ...        3.6644   
ENSG00000000005.6         0.5736        0.2720  ...        0.0120   
ENSG00000000419.13       38.9326       21.6648  ...       17.0009   
ENSG00000000457.14        3.5182        4.2998  ...        2.6672   
ENSG00000000460.17        1.0637        3.6655  ...        1.1932   

                    TCGA-UU-A93S  TCGA-V7-A7HQ  TCGA-W8-A86G  TCGA-WT-AB41  \
ENSG00000000003.15        2.5774       13.2320       26.6761       12.2652   
ENSG00000000005.6         0.8147        0.0887        0.3170        0.4446   
ENSG00000000419.13       36.1982       15.2455       30.7445       24.3154   
ENSG00000000457.14        4.8384        1.2055        5.0228        1.8221   
ENSG00000000460.17        2.8983        0.2148        1.4354        0.4214   

                    TCGA-WT-AB44  TCGA-XX-A899  TCGA-XX-A89A  TCGA-Z7-A8R5  \
ENSG00000000003.15        5.4684        9.2601        8.7338       16.0095   
ENSG00000000005.6         0.5633        0.2740       15.3096        0.2117   
ENSG00000000419.13       17.7092       26.6070       22.2972       27.1060   
ENSG00000000457.14        3.8705        4.2926        3.1576        3.2761   
ENSG00000000460.17        0.7080        1.3448        1.5348        0.5946   

                    TCGA-Z7-A8R6  
ENSG00000000003.15        9.0018  
ENSG00000000005.6         0.0767  
ENSG00000000419.13       34.8619  
ENSG00000000457.14        3.6285  
ENSG00000000460.17        3.9400  

[5 rows x 1076 columns]

In [ ]:
clinical_df = clinical_df.fillna(0)
column="Subtype_mRNA_PANCAN"
#for i in clinical_df[column].values:
#Print(i)
clinical_df = clinical_df.fillna(0)
column = "Subtype_mRNA_PANCAN"

# select the columns you want to output
selected_columns = ['sampleID', column]

# use .loc[] to select the rows and columns you want
clinical_df2 = clinical_df.loc[:, selected_columns]
print(clinical_df2.shape)
# select only the rows where neither column contains 0
clinical_df2 = clinical_df2.loc[(clinical_df2 != 0).all(axis=1)]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
new_label= le.fit_transform(clinical_df2["Subtype_mRNA_PANCAN"])
clinical_df2["Subtype_mRNA_PANCAN"] = new_label

# Remove the "-01" suffix from the sample names
clinical_df2["sampleID"] = clinical_df2["sampleID"].str.replace(r"-01$", "", regex=True)

# print the resulting DataFrame
clinical_df2

(1097, 2)


sampleID  Subtype_mRNA_PANCAN
0     TCGA-A1-A0SK                    0
1     TCGA-A1-A0SO                    0
2     TCGA-A1-A0SP                    0
3     TCGA-A2-A04P                    0
4     TCGA-A2-A04Q                    0
...            ...                  ...
1092  TCGA-AR-A0U1                    0
1093  TCGA-BH-A0AY                    2
1094  TCGA-C8-A1HF                    1
1095  TCGA-E2-A108                    4
1096  TCGA-E2-A1IP                    0

[1097 rows x 2 columns]

In [ ]:
collections.Counter(clinical_df["Subtype_mRNA_PANCAN"].values)

Counter({'Basal': 192, 'Her2': 82, 'LumA': 566, 'LumB': 217, 'Normal': 40})

In [ ]:
collections.Counter(clinical_df2["Subtype_mRNA_PANCAN"].values)

Counter({0: 192, 1: 82, 2: 566, 3: 217, 4: 40})

In [ ]:
clinical_df2 = clinical_df2.set_index('sampleID')
merged_df = rna_df.T.join(clinical_df2,how="inner")
merged_df

ENSG00000000003.15  ENSG00000000005.6  ENSG00000000419.13  \
TCGA-3C-AAAU              2.5997             0.0245             28.0941   
TCGA-3C-AALI              2.5597             0.0376             41.8377   
TCGA-3C-AALJ             13.1981             1.6187             48.7619   
TCGA-3C-AALK             12.8083             0.0530             22.4704   
TCGA-4H-AAAK             12.1947             0.2186             25.6404   
...                          ...                ...                 ...   
TCGA-WT-AB44              5.4684             0.5633             17.7092   
TCGA-XX-A899              9.2601             0.2740             26.6070   
TCGA-XX-A89A              8.7338            15.3096             22.2972   
TCGA-Z7-A8R5             16.0095             0.2117             27.1060   
TCGA-Z7-A8R6              9.0018             0.0767             34.8619   

              ENSG00000000457.14  ENSG00000000460.17  ENSG00000000938.13  \
TCGA-3C-AAAU              2.5028              1.0286              1.3283   
TCGA-3C-AALI              9.8346              2.2798              3.1867   
TCGA-3C-AALJ              2.5366              1.8393              5.0401   
TCGA-3C-AALK              3.7386              1.5078              2.2218   
TCGA-4H-AAAK              3.1496              1.3185              2.2639   
...                          ...                 ...                 ...   
TCGA-WT-AB44              3.8705              0.7080              4.4252   
TCGA-XX-A899              4.2926              1.3448              7.7138   
TCGA-XX-A89A              3.1576              1.5348              5.5952   
TCGA-Z7-A8R5              3.2761              0.5946              8.2715   
TCGA-Z7-A8R6              3.6285              3.9400              2.2662   

              ENSG00000000971.16  ENSG00000001036.14  ENSG00000001084.13  \
TCGA-3C-AAAU              2.0679              7.2405              3.3357   
TCGA-3C-AALI              6.1088             14.3510              5.2247   
TCGA-3C-AALJ              5.5963             12.4875              3.6519   
TCGA-3C-AALK              6.5522             10.6094              3.2970   
TCGA-4H-AAAK              8.5326             12.5900              3.6635   
...                          ...                 ...                 ...   
TCGA-WT-AB44              2.9825             19.5928              3.0310   
TCGA-XX-A899             11.3306             12.5267              4.7074   
TCGA-XX-A89A             12.2559             14.5091              4.4737   
TCGA-Z7-A8R5              6.9826             17.5916              2.4485   
TCGA-Z7-A8R6              1.9256              9.3586              2.4249   

              ENSG00000001167.14  ...  ENSG00000288654.1  ENSG00000288656.1  \
TCGA-3C-AAAU             16.4362  ...                0.0             0.0000   
TCGA-3C-AALI              9.1253  ...                0.0             0.0000   
TCGA-3C-AALJ              8.8129  ...                0.0             0.0000   
TCGA-3C-AALK             13.0935  ...                0.0             0.0050   
TCGA-4H-AAAK              9.2099  ...                0.0             0.0110   
...                          ...  ...                ...                ...   
TCGA-WT-AB44              8.0995  ...                0.0             0.0089   
TCGA-XX-A899              6.8561  ...                0.0             0.0052   
TCGA-XX-A89A              9.8494  ...                0.0             0.0057   
TCGA-Z7-A8R5              7.8471  ...                0.0             0.0000   
TCGA-Z7-A8R6             12.3490  ...                0.0             0.0000   

              ENSG00000288658.1  ENSG00000288660.1  ENSG00000288661.1  \
TCGA-3C-AAAU             0.0122             0.0747                0.0   
TCGA-3C-AALI             0.2715             0.0764                0.0   
TCGA-3C-AALJ             0.4181             0.1264                0.0   
TCGA-3C-AALK             0.0594             0.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

# prepare X and y
X = merged_df[merged_df.columns[:-1]].values
y = merged_df[merged_df.columns[-1]].values

# define random forest classifier
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features - this may take a while
feat_selector.fit(X, y)


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	19962
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	19962
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	19962
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	19962
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	19962
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	19962
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	19962
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	1562
Tentative: 	3152
Rejected: 	15248
Iteration: 	9 / 100
Confirmed: 	1562
Tentative: 	3152
Rejected: 	15248
Iteration: 	10 / 100
Confirmed: 	1562
Tentative: 	3152
Rejected: 	15248
Iteration: 	11 / 100
Confirmed: 	1562
Tentative: 	3152
Rejected: 	15248
Iteration: 	12 / 100
Confirmed: 	1639
Tentative: 	2336
Rejected: 	15987
Iteration: 	13 / 100
Confirmed: 	1639
Tentative: 	2336
Rejected: 	15987
Iteration: 	14 / 100
Confirmed: 	1639
Tentative: 	2336
Rejected: 	15987
Iteration: 	15 / 100
Confirmed: 	1639
Tenta

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=1332, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x77FBFB242F40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x77FBFB242F40, verbose=2)

In [ ]:
feat_selector.support_


array([False, False, False, ..., False, False, False])

In [ ]:
#feature_ranks = list(zip(merged_df.columns, feat_selector.ranking_,feat_selector.support_))

#feature_ranks.sort(key = lambda x:x[1])
#keeping_features = []

#for feat in feature_ranks:
    #print('Feature :{:<25} Rank: {}, keep: {}'.format(feat[0], feat[1], feat[2]))
    #if feat[1] < 10:
       # keeping_features.append(feat[0])


In [ ]:
feature_ranks = list(zip(merged_df.columns,
                        feat_selector.ranking_,feat_selector.support_))

feature_ranks.sort(key = lambda x:x[1])
keeping_features = []

i = 0

for index,  feat in enumerate(feature_ranks):
	if i < 1715:
		print('Feature :{:<25} Rank: {}, keep: {}'.format(feat[0], feat[1], feat[2]))
	i += 1
	if feat[1] < 1715:
		keeping_features.append(feat[0])

Feature :ENSG00000001617.12        Rank: 1, keep: True
Feature :ENSG00000002834.18        Rank: 1, keep: True
Feature :ENSG00000003096.14        Rank: 1, keep: True
Feature :ENSG00000003147.19        Rank: 1, keep: True
Feature :ENSG00000003989.18        Rank: 1, keep: True
Feature :ENSG00000004776.13        Rank: 1, keep: True
Feature :ENSG00000004809.14        Rank: 1, keep: True
Feature :ENSG00000004838.14        Rank: 1, keep: True
Feature :ENSG00000005102.14        Rank: 1, keep: True
Feature :ENSG00000005469.12        Rank: 1, keep: True
Feature :ENSG00000005513.10        Rank: 1, keep: True
Feature :ENSG00000006071.15        Rank: 1, keep: True
Feature :ENSG00000006128.12        Rank: 1, keep: True
Feature :ENSG00000006210.7         Rank: 1, keep: True
Feature :ENSG00000006377.11        Rank: 1, keep: True
Feature :ENSG00000006555.11        Rank: 1, keep: True
Feature :ENSG00000006625.18        Rank: 1, keep: True
Feature :ENSG00000006634.8         Rank: 1, keep: True
Feature :E

In [ ]:
len(keeping_features)

3582

In [ ]:
merged_df

ENSG00000000003.15  ENSG00000000005.6  ENSG00000000419.13  \
TCGA-3C-AAAU              2.5997             0.0245             28.0941   
TCGA-3C-AALI              2.5597             0.0376             41.8377   
TCGA-3C-AALJ             13.1981             1.6187             48.7619   
TCGA-3C-AALK             12.8083             0.0530             22.4704   
TCGA-4H-AAAK             12.1947             0.2186             25.6404   
...                          ...                ...                 ...   
TCGA-WT-AB44              5.4684             0.5633             17.7092   
TCGA-XX-A899              9.2601             0.2740             26.6070   
TCGA-XX-A89A              8.7338            15.3096             22.2972   
TCGA-Z7-A8R5             16.0095             0.2117             27.1060   
TCGA-Z7-A8R6              9.0018             0.0767             34.8619   

              ENSG00000000457.14  ENSG00000000460.17  ENSG00000000938.13  \
TCGA-3C-AAAU              2.5028              1.0286              1.3283   
TCGA-3C-AALI              9.8346              2.2798              3.1867   
TCGA-3C-AALJ              2.5366              1.8393              5.0401   
TCGA-3C-AALK              3.7386              1.5078              2.2218   
TCGA-4H-AAAK              3.1496              1.3185              2.2639   
...                          ...                 ...                 ...   
TCGA-WT-AB44              3.8705              0.7080              4.4252   
TCGA-XX-A899              4.2926              1.3448              7.7138   
TCGA-XX-A89A              3.1576              1.5348              5.5952   
TCGA-Z7-A8R5              3.2761              0.5946              8.2715   
TCGA-Z7-A8R6              3.6285              3.9400              2.2662   

              ENSG00000000971.16  ENSG00000001036.14  ENSG00000001084.13  \
TCGA-3C-AAAU              2.0679              7.2405              3.3357   
TCGA-3C-AALI              6.1088             14.3510              5.2247   
TCGA-3C-AALJ              5.5963             12.4875              3.6519   
TCGA-3C-AALK              6.5522             10.6094              3.2970   
TCGA-4H-AAAK              8.5326             12.5900              3.6635   
...                          ...                 ...                 ...   
TCGA-WT-AB44              2.9825             19.5928              3.0310   
TCGA-XX-A899             11.3306             12.5267              4.7074   
TCGA-XX-A89A             12.2559             14.5091              4.4737   
TCGA-Z7-A8R5              6.9826             17.5916              2.4485   
TCGA-Z7-A8R6              1.9256              9.3586              2.4249   

              ENSG00000001167.14  ...  ENSG00000288654.1  ENSG00000288656.1  \
TCGA-3C-AAAU             16.4362  ...                0.0             0.0000   
TCGA-3C-AALI              9.1253  ...                0.0             0.0000   
TCGA-3C-AALJ              8.8129  ...                0.0             0.0000   
TCGA-3C-AALK             13.0935  ...                0.0             0.0050   
TCGA-4H-AAAK              9.2099  ...                0.0             0.0110   
...                          ...  ...                ...                ...   
TCGA-WT-AB44              8.0995  ...                0.0             0.0089   
TCGA-XX-A899              6.8561  ...                0.0             0.0052   
TCGA-XX-A89A              9.8494  ...                0.0             0.0057   
TCGA-Z7-A8R5              7.8471  ...                0.0             0.0000   
TCGA-Z7-A8R6             12.3490  ...                0.0             0.0000   

              ENSG00000288658.1  ENSG00000288660.1  ENSG00000288661.1  \
TCGA-3C-AAAU             0.0122             0.0747                0.0   
TCGA-3C-AALI             0.2715             0.0764                0.0   
TCGA-3C-AALJ             0.4181             0.1264                0.0   
TCGA-3C-AALK             0.0594             0.

In [ ]:
#merged_df[keeping_features].to_csv("RNA seq_3582_reduced_features.csv")